In [2]:
#Importing Libraries
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten


In [3]:
base_dir = r'E:\Hope\AI Course Tamil\Week11-Deep Learning Module\Flower Detection\Dataset'
img_size = 180
batch = 32

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory( base_dir,
                                                       seed = 123,
                                                       validation_split=0.2,
                                                       subset = 'training',
                                                       batch_size=batch,
                                                       image_size=(img_size,img_size))

val_ds = tf.keras.utils.image_dataset_from_directory( base_dir,
                                                       seed = 123,
                                                       validation_split=0.2,
                                                       subset = 'validation',
                                                       batch_size=batch,
                                                       image_size=(img_size,img_size))

Found 4318 files belonging to 5 classes.
Using 3455 files for training.
Found 4318 files belonging to 5 classes.
Using 863 files for validation.


In [5]:
flower_names = train_ds.class_names
flower_names

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

In [7]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [8]:
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [9]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal", input_shape = (img_size,img_size,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [10]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5)
])

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

In [13]:
history = model.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
108/108 [==============================] - 92s 711ms/step - loss: 1.3628 - accuracy: 0.4113 - val_loss: 1.1191 - val_accuracy: 0.5377
Epoch 2/15
108/108 [==============================] - 79s 728ms/step - loss: 1.0823 - accuracy: 0.5705 - val_loss: 1.0300 - val_accuracy: 0.5910
Epoch 3/15
108/108 [==============================] - 79s 730ms/step - loss: 0.9776 - accuracy: 0.6211 - val_loss: 0.9642 - val_accuracy: 0.6408
Epoch 4/15
108/108 [==============================] - 76s 708ms/step - loss: 0.9249 - accuracy: 0.6428 - val_loss: 0.9880 - val_accuracy: 0.6327
Epoch 5/15
108/108 [==============================] - 78s 724ms/step - loss: 0.8540 - accuracy: 0.6590 - val_loss: 0.8713 - val_accuracy: 0.6489
Epoch 6/15
108/108 [==============================] - 75s 698ms/step - loss: 0.8086 - accuracy: 0.6831 - val_loss: 0.8694 - val_accuracy: 0.6721
Epoch 7/15
108/108 [==============================] - 79s 733ms/step - loss: 0.7762 - accuracy: 0.6883 - val_loss: 0.8884 - val_ac

In [16]:
def classify_images(image_path):
    input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array,0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    outcome = 'The Image belongs to ' + flower_names[np.argmax(result)] + ' with a score of '+ str(np.max(result)*100)
    return outcome

In [20]:
classify_images('sample/rose.jpg')

1/1 [==============================] - 1s 1s/step


'The Image belongs to rose with a score of 85.52783727645874'

In [21]:
from tensorflow.keras.models import load_model 
model.save('Flower_recognition_Model.h5') 



In [ ]:
import tensorflow as tf

# Load your pre-trained model
model = tf.keras.models.load_model('Flower_recognition_Model.h5')

def classify_images(image_path):
    input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
    input_image_array = tf.keras.utils.img_to_array(input_image)
    input_image_exp_dim = tf.expand_dims(input_image_array,0)

    predictions = model.predict(input_image_exp_dim)
    result = tf.nn.softmax(predictions[0])
    outcome = 'The Image belongs to ' + flower_names[np.argmax(result)] + ' with a score of '+ str(np.max(result)*100)

    # Display the image
    plt.figure(figsize=(6,6))
    plt.imshow(input_image_array.astype(np.uint8))
    plt.title(outcome)
    plt.axis('off')
    plt.show()

    return outcome

flower_names = ['Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip']

outcome = classify_images('sample/rose.jpg')
print(outcome)


1/1 [==============================] - 1s 552ms/step
